<a href="https://colab.research.google.com/github/Viniciusp67/Consultor-de-Direito-Eleitoral/blob/main/teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.2 MB/s eta 0:00:00


In [ ]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
enc.encode('realizando testes')

[5305, 528, 25440, 1332, 274]

In [ ]:
texto = 'A soberania popular será exercida pelo sufrágio universal e pelo voto direto e secreto, com valor igual para todos...'
alfabeto = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
caracteres = sorted(list(set(texto + alfabeto)))
print(caracteres)
print('Tamanho do seu vocabulário >> ', len(caracteres))

[' ', ',', '.', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'á']
Tamanho do seu vocabulário >>  56


In [ ]:
letraParaIndex = { lt:i for i,lt in enumerate(caracteres) }
IndexParaLetra = { i:lt for i,lt in enumerate(caracteres) }

encode = lambda s: [letraParaIndex[c] for c in s]
decode = lambda l: ''.join([IndexParaLetra[i] for i in l])

In [ ]:
print(encode('A soberania popular será exercida pelo sufrágio universal e pelo voto direto e secreto, com valor igual para todos...'))

[3, 0, 47, 43, 30, 33, 46, 29, 42, 37, 29, 0, 44, 43, 44, 49, 40, 29, 46, 0, 47, 33, 46, 55, 0, 33, 52, 33, 46, 31, 37, 32, 29, 0, 44, 33, 40, 43, 0, 47, 49, 34, 46, 55, 35, 37, 43, 0, 49, 42, 37, 50, 33, 46, 47, 29, 40, 0, 33, 0, 44, 33, 40, 43, 0, 50, 43, 48, 43, 0, 32, 37, 46, 33, 48, 43, 0, 33, 0, 47, 33, 31, 46, 33, 48, 43, 1, 0, 31, 43, 41, 0, 50, 29, 40, 43, 46, 0, 37, 35, 49, 29, 40, 0, 44, 29, 46, 29, 0, 48, 43, 32, 43, 47, 2, 2, 2]


In [ ]:
texto = encode('Compra de votos e outras formas de influenciar ilegalmente o eleitor')
len(texto)
print(texto)


[5, 43, 41, 44, 46, 29, 0, 32, 33, 0, 50, 43, 48, 43, 47, 0, 33, 0, 43, 49, 48, 46, 29, 47, 0, 34, 43, 46, 41, 29, 47, 0, 32, 33, 0, 37, 42, 34, 40, 49, 33, 42, 31, 37, 29, 46, 0, 37, 40, 33, 35, 29, 40, 41, 33, 42, 48, 33, 0, 43, 0, 33, 40, 33, 37, 48, 43, 46]


In [ ]:
for i in range(len(texto)-1):
  x = texto[:i]
  y = texto[i]
  if x != []:
    print(f'Quando os dados forem: {x} o alvo é {y}')

In [ ]:
from random import randint

for i in range(len(texto)):
  x = texto
  y = texto.copy()
  idx_mask = randint(0,len(texto)-1)
  y[idx_mask] = '<mask>'
  print(f'Quando os dados forem: {x} o alvo é {y}')

In [ ]:
!pip install transformers[torch]

In [ ]:
PATH = './sample_data/'
dados_treino = 'Codigo_Eleitoral.txt'

In [ ]:
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=[PATH+dados_treino], vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
    ])

In [ ]:
!rm -r ./sample_data/RAW_MODEL
!mkdir ./sample_data/RAW_MODEL
tokenizer.save_model(PATH+'RAW_MODEL')

rm: cannot remove './sample_data/RAW_MODEL': No such file or directory


['./sample_data/RAW_MODEL/vocab.json', './sample_data/RAW_MODEL/merges.txt']

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    PATH+'RAW_MODEL'+"/vocab.json",
    PATH+'RAW_MODEL'+"/merges.txt",
)

tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
from transformers import RobertaTokenizer

tokenizer =  RobertaTokenizer.from_pretrained(PATH+'RAW_MODEL', max_len=512)

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=512,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config)

In [ ]:
model.num_parameters()

83502880

In [ ]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=PATH+dados_treino,
    block_size=128,
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
dataset.examples[:2]

[{'input_ids': tensor([   0,  387,   18,  307,  284, 4725,  816,  594,  820, 1424, 4816,  262,
          4849,  262, 2165,  263,  278,  997,  265, 2468, 1197,  448,  295,  386,
          4269,  364,  265,  883,  263,  357, 2028,   18,    2])},
 {'input_ids': tensor([   0,  597,  587,   18,  478,  426,  659,  517,  962,  455, 1561,  355,
           619, 5517, 1510,   18,    2])}]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.1
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=PATH+'RAW_MODEL',
    overwrite_output_dir=True,
    num_train_epochs=200,
    per_device_train_batch_size=64,
    save_steps=50,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
trainer.train()

Step,Training Loss
